### Task 2

Antal län i dataset

In [9]:
import pandas as pd
import plotly.express as px

vaccine_data= pd.read_excel("Labb-data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name="Vaccinerade kommun och ålder")

antal_lan = vaccine_data["Län_namn"].nunique()

print(f"Antal län representerade i datasetet: {antal_lan}")

Antal län representerade i datasetet: 21


### Antal kommuner i datsetet 

In [10]:
antal_kommuner = vaccine_data["Kommun_namn"].nunique()
print(f"Antal kommuner representerade i datasetet: {antal_kommuner}")


Antal kommuner representerade i datasetet: 290


### Antal befolkning

In [11]:
antal_befolkning = vaccine_data["Befolkning"].sum()
print(f"Antal befolkning som är med i datasetet: {antal_befolkning} personer")

Antal befolkning som är med i datasetet: 9092790 personer


### Antal barn med i datasetet
Antal barn i sverige mellan 12-18 år är runt 779 000, enligt datsetet så är 745 370 av dem vaccinerade (källa: scb)

In [12]:
def extract_lower_bound(age_range):
    if "eller" in age_range:
        return 90  
    return int(age_range.split("-")[0])

vaccine_data["Lägsta_ålder"] = vaccine_data["Ålder"].apply(extract_lower_bound)


filtered_df = vaccine_data[vaccine_data["Lägsta_ålder"] < 18]

total_population_under_18 = filtered_df["Befolkning"].sum()

print("Antal befolkning under 18:", total_population_under_18)


Antal befolkning under 18: 745370


### Åldersföredling av svergies befolkning

In [13]:
grouped_df = vaccine_data.groupby("Ålder")["Befolkning"].sum().reset_index()

fig = px.bar(grouped_df, x="Ålder", y="Befolkning", color="Ålder", labels={"Befolkning": "Antal"},
             title="Total befolkning per åldersgrupp")

fig.show()
# fig.write_html("Task2e.html")

### Stapeldiagram för Antal vaccinerade i olika län

- To get all län, use .unique and a for-loop to itterate over unique län_namn column.

In [14]:
selected_lan = ["Stockholms län", "Västra Götalands län", "Skåne län"]
selected_data = vaccine_data[vaccine_data["Län_namn"].isin(selected_lan)]

lan_groups = selected_data.groupby(["Län_namn"]).agg({
    "Befolkning": "sum",
    "Andel minst 1 dos": "mean",  
    "Andel minst 2 doser": "mean",  
    "Andel 3 doser": "mean"  
})

lan_groups = lan_groups.reset_index()

# Multiply by 100 to get percentages
lan_groups["Andel minst 1 dos"] *= 100
lan_groups["Andel minst 2 doser"] *= 100
lan_groups["Andel 3 doser"] *= 100


melted_df = pd.melt(lan_groups, id_vars=["Län_namn"], 
                    value_vars=["Andel minst 1 dos", "Andel minst 2 doser", "Andel 3 doser"],
                    var_name="Dos", value_name="Andel")

fig = px.bar(melted_df,
             x="Län_namn",
             y="Andel",
             color="Dos",
             barmode="group",
             labels={"Andel": "Andel i procent %", "Dos": "Dos", "Län_namn": "Län"},
             title="Andel vaccinerade med dos 1, dos 2 och dos 3 efter län",
             template="plotly_dark"
             )

fig.show()
# fig.write_html("Task2f.html")

### Jämför vaccinerade i Västra götalands län och Stockholms län

In [15]:
selected_lan = ["Stockholms län", "Västra Götalands län"]
selected_data = vaccine_data[vaccine_data["Län_namn"].isin(selected_lan)].copy()  # Create a copy to avoid SettingWithCopyWarning


selected_data["Antal 3 doser"].fillna(0, inplace=True)
selected_data["Antal 4 doser"].fillna(0, inplace=True)


selected_data.loc[:, "Andel minst 1 dos"] = selected_data["Antal minst 1 dos"] / selected_data["Befolkning"] * 100
selected_data.loc[:, "Andel minst 2 doser"] = selected_data["Antal minst 2 doser"] / selected_data["Befolkning"] * 100
selected_data.loc[:, "Andel 3 doser"] = selected_data["Antal 3 doser"] / selected_data["Befolkning"] * 100
selected_data.loc[:, "Andel 4 doser"] = selected_data["Antal 4 doser"] / selected_data["Befolkning"] * 100

lan_groups = selected_data.groupby(["Län_namn"]).agg({
    "Befolkning": "sum",
    "Andel minst 1 dos": "mean",
    "Andel minst 2 doser": "mean",
    "Andel 3 doser": "mean",
    "Andel 4 doser": "mean"
}).reset_index()


melted_lan = pd.melt(lan_groups, id_vars=["Län_namn"], 
                     value_vars=["Andel minst 1 dos", "Andel minst 2 doser", "Andel 3 doser", "Andel 4 doser"],
                     var_name="Dos", value_name="Andel")

fig = px.bar(melted_lan,
             x="Län_namn",
             y="Andel",
             color="Dos",
             barmode="group",
             labels={"Andel": "Andel i procent %", "Dos": "Dos", "Län_namn": "Län"},
             title="Andel vaccinerade med dos 1, dos 2, dos 3 och dos 4 efter län",
             template="plotly_dark"
             )

fig.show()
# fig.write_html("Task2g.html")
